# Lesson 3: Multiple Tools & Context

This lesson demonstrates how agents select and orchestrate multiple tools.

## Concepts Covered
- Tool selection logic based on descriptions
- Multiple tool registration with `tools=[tool1, tool2, tool3]`
- Tool orchestration and chaining in single response
- Complex reasoning chains with multiple steps
- Tool interdependencies and combined results
- Context-aware tool selection

## Learning Goals
- ☐ Agent selects appropriate tool based on query intent
- ☐ Can chain multiple tools in one response seamlessly
- ☐ Handles ambiguous requests using conversation context
- ☐ Combines tool results into coherent responses
- ☐ Tool selection is accurate even with overlapping capabilities

## Setup: Import Dependencies

In [1]:
from datetime import datetime
from zoneinfo import ZoneInfo
from strands import Agent, tool

# Import shared utilities for model configuration
from lesson_utils import (
    load_environment,
    create_working_model,
    check_api_keys,
)

# Load environment and check API keys
load_environment()
check_api_keys()

✅ API keys detected: OpenAI


True

## Tool 1: Weather Tool

A simple weather tool that returns mock weather data for major cities.

In [2]:
@tool
def get_weather(city: str) -> str:
    """Get current weather conditions for a city.

    Args:
        city: The name of the city to get weather for

    Returns:
        Current weather conditions including temperature and description
    """
    # Mock weather data for demonstration
    weather_data = {
        "Tokyo": {"temp": 22, "condition": "Partly cloudy", "humidity": 65},
        "New York": {"temp": 18, "condition": "Rainy", "humidity": 80},
        "London": {"temp": 15, "condition": "Foggy", "humidity": 75},
        "Paris": {"temp": 20, "condition": "Sunny", "humidity": 55},
        "Sydney": {"temp": 25, "condition": "Clear", "humidity": 50},
        "Singapore": {"temp": 30, "condition": "Hot and humid", "humidity": 85},
        "San Francisco": {"temp": 16, "condition": "Partly cloudy", "humidity": 70},
    }

    city_normalized = city.strip()

    for known_city, data in weather_data.items():
        if known_city.lower() == city_normalized.lower():
            return (
                f"Weather in {known_city}: {data['condition']}, "
                f"{data['temp']}°C, {data['humidity']}% humidity"
            )

    return f"Weather data not available for {city}. Available cities: {', '.join(weather_data.keys())}"

## Tool 2: Time/Timezone Tool

Returns current time for major cities using Python's `zoneinfo` module.

In [3]:
@tool
def get_time(city: str) -> str:
    """Get current time for a city.

    Args:
        city: The name of the city to get time for

    Returns:
        Current local time and timezone information
    """
    city_timezones = {
        "Tokyo": "Asia/Tokyo",
        "New York": "America/New_York",
        "London": "Europe/London",
        "Paris": "Europe/Paris",
        "Sydney": "Australia/Sydney",
        "Singapore": "Asia/Singapore",
        "San Francisco": "America/Los_Angeles",
    }

    city_normalized = city.strip()

    for known_city, tz_name in city_timezones.items():
        if known_city.lower() == city_normalized.lower():
            try:
                tz = ZoneInfo(tz_name)
                local_time = datetime.now(tz)
                time_str = local_time.strftime("%I:%M %p")
                date_str = local_time.strftime("%A, %B %d, %Y")

                return (
                    f"Current time in {known_city}: {time_str} ({tz_name})\n"
                    f"Date: {date_str}"
                )
            except Exception as e:
                return f"Error getting time for {known_city}: {str(e)}"

    return f"Time data not available for {city}. Available cities: {', '.join(city_timezones.keys())}"

## Tool 3: Unit Converter Tool

Converts between temperature, distance, and weight units.

In [4]:
@tool
def convert_units(value: float, from_unit: str, to_unit: str) -> str:
    """Convert between different units of measurement.

    Supports temperature (C, F, K), distance (m, km, mi, ft), and weight (kg, lb, oz).

    Args:
        value: The numeric value to convert
        from_unit: The source unit (e.g., 'C', 'F', 'km', 'kg')
        to_unit: The target unit (e.g., 'F', 'C', 'mi', 'lb')

    Returns:
        Converted value with units
    """
    from_unit = from_unit.lower().strip()
    to_unit = to_unit.lower().strip()

    # Temperature conversions
    if from_unit in ['c', 'celsius', 'f', 'fahrenheit', 'k', 'kelvin']:
        if from_unit in ['f', 'fahrenheit']:
            celsius = (value - 32) * 5/9
        elif from_unit in ['k', 'kelvin']:
            celsius = value - 273.15
        else:
            celsius = value

        if to_unit in ['f', 'fahrenheit']:
            result = celsius * 9/5 + 32
            return f"{value}°{from_unit.upper()} = {result:.1f}°F"
        elif to_unit in ['k', 'kelvin']:
            result = celsius + 273.15
            return f"{value}°{from_unit.upper()} = {result:.1f}K"
        elif to_unit in ['c', 'celsius']:
            return f"{value}°{from_unit.upper()} = {celsius:.1f}°C"
        else:
            return f"Unknown temperature unit: {to_unit}. Use C, F, or K."

    # Distance conversions
    elif from_unit in ['m', 'meter', 'meters', 'km', 'kilometer', 'kilometers',
                       'mi', 'mile', 'miles', 'ft', 'feet', 'foot']:
        if from_unit in ['km', 'kilometer', 'kilometers']:
            meters = value * 1000
        elif from_unit in ['mi', 'mile', 'miles']:
            meters = value * 1609.34
        elif from_unit in ['ft', 'feet', 'foot']:
            meters = value * 0.3048
        else:
            meters = value

        if to_unit in ['km', 'kilometer', 'kilometers']:
            result = meters / 1000
            return f"{value} {from_unit} = {result:.2f} km"
        elif to_unit in ['mi', 'mile', 'miles']:
            result = meters / 1609.34
            return f"{value} {from_unit} = {result:.2f} miles"
        elif to_unit in ['ft', 'feet', 'foot']:
            result = meters / 0.3048
            return f"{value} {from_unit} = {result:.1f} feet"
        elif to_unit in ['m', 'meter', 'meters']:
            return f"{value} {from_unit} = {meters:.2f} meters"
        else:
            return f"Unknown distance unit: {to_unit}. Use m, km, mi, or ft."

    # Weight conversions
    elif from_unit in ['kg', 'kilogram', 'kilograms', 'lb', 'pound', 'pounds',
                       'oz', 'ounce', 'ounces']:
        if from_unit in ['lb', 'pound', 'pounds']:
            kg = value * 0.453592
        elif from_unit in ['oz', 'ounce', 'ounces']:
            kg = value * 0.0283495
        else:
            kg = value

        if to_unit in ['lb', 'pound', 'pounds']:
            result = kg / 0.453592
            return f"{value} {from_unit} = {result:.2f} lb"
        elif to_unit in ['oz', 'ounce', 'ounces']:
            result = kg / 0.0283495
            return f"{value} {from_unit} = {result:.2f} oz"
        elif to_unit in ['kg', 'kilogram', 'kilograms']:
            return f"{value} {from_unit} = {kg:.2f} kg"
        else:
            return f"Unknown weight unit: {to_unit}. Use kg, lb, or oz."

    else:
        return (
            f"Unknown unit type: {from_unit}. Supported categories:\n"
            "- Temperature: C, F, K\n"
            "- Distance: m, km, mi, ft\n"
            "- Weight: kg, lb, oz"
        )

## Part 1: Single Tool Selection

Demonstrate agent selecting the appropriate tool based on query intent.

In [5]:
working_model = create_working_model("tool selection")

# Create agent with all three tools
agent = Agent(
    model=working_model,
    tools=[get_weather, get_time, convert_units],
    system_prompt="You are a helpful assistant with access to weather, time, and unit conversion tools."
)

# Test: Weather query
print("Query: What's the weather in Tokyo?")
agent("What's the weather in Tokyo?")

🚀 Using OpenAI gpt-4o-mini for tool selection (precise mode)
Query: What's the weather in Tokyo?

Tool #1: get_weather
The weather in Tokyo is partly cloudy with a temperature of 22°C and 65% humidity.

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': 'The weather in Tokyo is partly cloudy with a temperature of 22°C and 65% humidity.'}]}, metrics=EventLoopMetrics(cycle_count=2, tool_metrics={'get_weather': ToolMetrics(tool={'toolUseId': 'call_1tOeoYAEAtolil03Dju3VaJq', 'name': 'get_weather', 'input': {'city': 'Tokyo'}}, call_count=1, success_count=1, error_count=0, total_time=0.0004200935363769531)}, cycle_durations=[3.3708879947662354], traces=[<strands.telemetry.metrics.Trace object at 0x114f2f490>, <strands.telemetry.metrics.Trace object at 0x11554e410>], accumulated_usage={'inputTokens': 694, 'outputTokens': 35, 'totalTokens': 729}, accumulated_metrics={'latencyMs': 0}), state={})

In [6]:
# Test: Time query
print("Query: What time is it in New York?")
agent("What time is it in New York?")

Query: What time is it in New York?

Tool #2: get_time
The current time in New York is 11:01 AM on Saturday, October 4, 2025.

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': 'The current time in New York is 11:01 AM on Saturday, October 4, 2025.'}]}, metrics=EventLoopMetrics(cycle_count=4, tool_metrics={'get_weather': ToolMetrics(tool={'toolUseId': 'call_1tOeoYAEAtolil03Dju3VaJq', 'name': 'get_weather', 'input': {'city': 'Tokyo'}}, call_count=1, success_count=1, error_count=0, total_time=0.0004200935363769531), 'get_time': ToolMetrics(tool={'toolUseId': 'call_1vLQ3OymqqAqqckDeHTTgbhT', 'name': 'get_time', 'input': {'city': 'New York'}}, call_count=1, success_count=1, error_count=0, total_time=0.0019278526306152344)}, cycle_durations=[3.3708879947662354, 4.377479076385498], traces=[<strands.telemetry.metrics.Trace object at 0x114f2f490>, <strands.telemetry.metrics.Trace object at 0x11554e410>, <strands.telemetry.metrics.Trace object at 0x114715000>, <strands.telemetry.metrics.Trace object at 0x114f2ea70>], accumulated_usage={'inputTokens': 1549, 'outputTokens': 74, 'total

In [7]:
# Test: Unit conversion query
print("Query: Convert 100 km to miles")
agent("Convert 100 km to miles")

Query: Convert 100 km to miles

Tool #3: convert_units
100 kilometers is equal to 62.14 miles.

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': '100 kilometers is equal to 62.14 miles.'}]}, metrics=EventLoopMetrics(cycle_count=6, tool_metrics={'get_weather': ToolMetrics(tool={'toolUseId': 'call_1tOeoYAEAtolil03Dju3VaJq', 'name': 'get_weather', 'input': {'city': 'Tokyo'}}, call_count=1, success_count=1, error_count=0, total_time=0.0004200935363769531), 'get_time': ToolMetrics(tool={'toolUseId': 'call_1vLQ3OymqqAqqckDeHTTgbhT', 'name': 'get_time', 'input': {'city': 'New York'}}, call_count=1, success_count=1, error_count=0, total_time=0.0019278526306152344), 'convert_units': ToolMetrics(tool={'toolUseId': 'call_UpeYtDIDStOPRAFya1xnrfE8', 'name': 'convert_units', 'input': {'value': 100, 'from_unit': 'km', 'to_unit': 'mi'}}, call_count=1, success_count=1, error_count=0, total_time=0.0005958080291748047)}, cycle_durations=[3.3708879947662354, 4.377479076385498, 3.64266300201416], traces=[<strands.telemetry.metrics.Trace object at 0x114f2f490>, <s

## Part 2: Tool Chaining

Demonstrate agent using multiple tools in one response.

In [8]:
working_model = create_working_model("tool chaining")

agent = Agent(
    model=working_model,
    tools=[get_weather, get_time, convert_units],
    system_prompt="You are a helpful assistant. Use multiple tools when needed to give complete answers."
)

# Query requiring multiple tools
print("Query: What's the weather and time in London?")
agent("What's the weather and time in London?")

🚀 Using OpenAI gpt-4o-mini for tool chaining (precise mode)
Query: What's the weather and time in London?

Tool #1: get_weather

Tool #2: get_time
The current weather in London is foggy with a temperature of 15°C and 75% humidity. 

The local time in London is 04:01 PM on Saturday, October 4, 2025.

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': 'The current weather in London is foggy with a temperature of 15°C and 75% humidity. \n\nThe local time in London is 04:01 PM on Saturday, October 4, 2025.'}]}, metrics=EventLoopMetrics(cycle_count=2, tool_metrics={'get_weather': ToolMetrics(tool={'toolUseId': 'call_cFWGyaJHkFYDsJ7BnQq5YOFb', 'name': 'get_weather', 'input': {'city': 'London'}}, call_count=1, success_count=1, error_count=0, total_time=0.0008168220520019531), 'get_time': ToolMetrics(tool={'toolUseId': 'call_EWm5YI2FO0HNQCUNMZUHjrll', 'name': 'get_time', 'input': {'city': 'London'}}, call_count=1, success_count=1, error_count=0, total_time=0.0010962486267089844)}, cycle_durations=[1.8081998825073242], traces=[<strands.telemetry.metrics.Trace object at 0x1155a1150>, <strands.telemetry.metrics.Trace object at 0x114f2eef0>], accumulated_usage={'inputTokens': 762, 'outputTokens': 89, 'totalTokens': 851}, accumulated_metrics={'latencyMs': 0}

In [ ]:
# Complex query with 3 tools
print("Query: Can you tell me the current temperature in Paris (°F)?")
agent("Can you tell me the current temperature in Paris (°F)?")

Query: Tell me about Paris - weather, time, and convert 20°C to Fahrenheit

Tool #13: get_weather

Tool #14: convert_units
The current temperature in Paris is 68°F.

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': 'The current temperature in Paris is 68°F.'}]}, metrics=EventLoopMetrics(cycle_count=15, tool_metrics={'get_weather': ToolMetrics(tool={'toolUseId': 'call_4Iyxk0sJKcuLCwaFFpVxBTIU', 'name': 'get_weather', 'input': {'city': 'Paris'}}, call_count=6, success_count=6, error_count=0, total_time=0.005405902862548828), 'get_time': ToolMetrics(tool={'toolUseId': 'call_02UHxWnesLuVhxVeRzjUUn56', 'name': 'get_time', 'input': {'city': 'Paris'}}, call_count=3, success_count=3, error_count=0, total_time=0.0034084320068359375), 'convert_units': ToolMetrics(tool={'toolUseId': 'call_RFkLyBPIPh5mwHOjOEa7Wka7', 'name': 'convert_units', 'input': {'value': 20, 'from_unit': 'C', 'to_unit': 'F'}}, call_count=5, success_count=5, error_count=0, total_time=0.0031807422637939453)}, cycle_durations=[1.8081998825073242, 1.7210540771484375, 1.6255648136138916, 0.8587899208068848, 1.0631332397460938, 0.8769350051879883], traces=[

## Part 3: Context-Aware Tool Selection

Demonstrate agent using context from conversation.

In [ ]:
working_model = create_working_model("context awareness")

agent = Agent(
    model=working_model,
    tools=[get_weather, get_time, convert_units],
    system_prompt="You are a helpful assistant. Remember context from previous messages."
)

# Multi-turn conversation
print("Turn 1: What's the weather in Tokyo?")
agent("What's the weather in Tokyo?")

print("\nTurn 2: What about Sydney? (ambiguous - should infer 'weather')")
agent("What about Sydney?")

print("\nTurn 3: And the time there? (should know 'there' = Sydney)")
agent("And the time there?")

## Part 4: Error Handling

Demonstrate tools handling errors gracefully.

In [ ]:
working_model = create_working_model("error handling")

agent = Agent(
    model=working_model,
    tools=[get_weather, get_time, convert_units],
    system_prompt="You are a helpful assistant. When tools return errors, explain them clearly."
)

# Test: Non-existent city
print("Query: What's the weather in Atlantis?")
agent("What's the weather in Atlantis?")

In [ ]:
# Test: Invalid unit
print("Query: Convert 100 meters to bananas")
agent("Convert 100 meters to bananas")

## Part 5: Combined Reasoning with Multiple Tools

Demonstrate complex queries requiring reasoning and multiple tool calls.

In [ ]:
working_model = create_working_model("combined reasoning")

agent = Agent(
    model=working_model,
    tools=[get_weather, get_time, convert_units],
    system_prompt="You are a helpful travel assistant. Combine tool information with reasoning."
)

# Complex query requiring multiple tools and reasoning
query = (
    "I'm in New York and want to call someone in Singapore. "
    "What's the weather like there, what time is it, and is it a good time to call?"
)

print(f"Query: {query}")
agent(query)

## Success Criteria

After completing this lesson, you should have seen:

- ☐ Agent selects appropriate tool based on query intent
  - Weather queries use get_weather
  - Time queries use get_time
  - Conversion queries use convert_units

- ☐ Can chain multiple tools in one response seamlessly
  - "Weather and time in London" uses both tools
  - Results are combined coherently

- ☐ Handles ambiguous requests using conversation context
  - "What about Sydney?" infers weather from previous question
  - "And the time there?" knows "there" refers to Sydney

- ☐ Combines tool results into coherent responses
  - Tool outputs are integrated into natural language
  - Agent adds reasoning and interpretation

- ☐ Tool selection is accurate even with overlapping capabilities
  - Doesn't confuse temperature conversion with weather
  - Selects correct tool even when multiple could apply

## Experiments to Try

Try these experiments in new cells below:

1. **Multi-city comparison:**
   - "Compare weather between Tokyo, New York, and London"

2. **Complex unit conversion chains:**
   - "Convert 5 km to miles, then to feet"

3. **Time zone calculations:**
   - "If it's 3pm in San Francisco, what time is it in Singapore?"

4. **Add a new tool:**
   - Create currency converter
   - Add to tools list
   - Test tool orchestration with 4 tools

5. **Test ambiguous context:**
   - Start conversation about Paris
   - Ask follow-ups without mentioning city name
   - See if agent maintains context

6. **Error recovery:**
   - Give intentionally bad inputs
   - See how agent explains errors
   - Test if agent suggests corrections

## Next Steps

Continue to **Lesson 4: Stateful Tools with Classes** to learn how to create tools that maintain state across invocations.